In [1]:
!pip install streamlit
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 117.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.5 MB/s eta 0:00:00


In [2]:
import torch
import pandas as pd
import streamlit as st
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import json
import os
from datasets import Dataset
import numpy as np

# Disable Weights & Biases logging
os.environ["WANDB_DISABLED"] = "true"

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load manually uploaded ChemProt dataset
train_path = "/content/train.jsonl"
test_path = "/content/test.jsonl"

def load_jsonl_data(file_path):
    with open(file_path, "r") as f:
        return [json.loads(line) for line in f]

train_data = load_jsonl_data(train_path)
test_data = load_jsonl_data(test_path)
print("Dataset loaded successfully!")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("michiyasunaga/BioLinkBERT-base")

# Map labels to numeric values
unique_labels = list(set(d['label'] for d in train_data))
label_to_id = {label: i for i, label in enumerate(unique_labels)}
id_to_label = {i: label for label, i in label_to_id.items()}

def preprocess_data(examples):
    inputs = tokenizer([ex['text'] for ex in examples], padding=True, truncation=True, max_length=512)
    labels = [label_to_id[ex['label']] for ex in examples]
    inputs['labels'] = labels
    return inputs

train_encodings = preprocess_data(train_data)
test_encodings = preprocess_data(test_data)

train_dataset = Dataset.from_dict(train_encodings)
test_dataset = Dataset.from_dict(test_encodings)

model = AutoModelForSequenceClassification.from_pretrained("michiyasunaga/BioLinkBERT-base", num_labels=len(unique_labels)).to(device)

# Training arguments
training_args = TrainingArguments(
    output_dir="/content/results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

# Evaluate model
def compute_metrics(pred):
    labels = np.array(pred.label_ids)  # Ensure labels are a NumPy array
    preds = np.argmax(pred.predictions, axis=1)  # Take the highest probability class
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = accuracy_score(labels, preds)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Get predictions
eval_results = trainer.predict(test_dataset)
eval_metrics = compute_metrics(eval_results)
print("Evaluation Metrics:", eval_metrics)

# Save the trained model
model.save_pretrained("/content/chemprot_relation_model")
tokenizer.save_pretrained("/content/chemprot_relation_model")

# Streamlit Web UI
st.title("Biomedical Relation Extraction")
st.write("Enter biomedical text and extract relations between entities.")

# Input field
text_input = st.text_area("Enter biomedical text:", "")

# Entity Selection
types = ["Gene", "Protein", "Disease", "Drug"]
selected_entity = st.selectbox("Select entity type:", types)

# Extract Relations
if st.button("Extract Relations"):
    if text_input.strip():
        inputs = tokenizer(text_input, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
        predicted_label = torch.argmax(logits, dim=1).item()
        relation = id_to_label[predicted_label]
        st.success(f"Extracted Relation: {relation}")
    else:
        st.error("Please enter some text.")


Using device: cuda
Dataset loaded successfully!


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/379 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/225k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/447k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss
1,0.848600,0.542008
2,0.371000,0.570972
3,0.223000,0.586251


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Evaluation Metrics: {'accuracy': 0.8659556068031133, 'precision': 0.8525670469968832, 'recall': 0.8659556068031133, 'f1': 0.8579941091041737}


2025-03-14 15:59:44.624 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-14 15:59:45.023 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-03-14 15:59:45.024 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-14 15:59:45.025 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-14 15:59:45.026 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-14 15:59:45.027 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-14 15:59:45.027 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-14 15:59:45.028 Thread 'MainThread': mi